## Establishing a baseline

For the project we needed to establish a baseline to compare our results to. We achieve this on both Hate speech version two dataset, and the Toxify dataset.

In [12]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from src.preprocessing.hatespeech_dataset_querying import hatespeech_v2_load_train_and_validation_set

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../..")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Dir has already been changed


## Hate speech version 2

In [13]:
train_df, validation_df = hatespeech_v2_load_train_and_validation_set()
train_df

,tweet_id,text,label,topic
0,1376967244706553858,"Father Brennan : When the Jews return to Zion,...",0,0.0
1,1344315766594707456,"On December 30, 1767, in Savannah, Georgia, sl...",0,2.0
2,1233898243731251201,Russian hackers are still going to be a huge p...,0,2.0
3,1277958772116254721,Let me be clear whatever you have to say about...,1,4.0
4,1344535608018886657,In dorana district of Mp hindutava organizatio...,0,0.0
...,...,...,...,...
43896,1377295513238732803,Intention to create fear for social/political ...,0,2.0
43897,1277277939093078016,A wise person from my country of historic orig...,1,0.0
43898,1344604336572215296,"""Remember Jesus Christ, risen from the dead, d...",0,0.0
43899,1277988673812754432,honestly this shit pisses me off and is just s...,1,3.0


In [14]:
def print_label_counts(df):
    ## predicting the majority label 
    label_distribution = df["label"].value_counts().to_frame().reset_index() 
    label_distribution["count"] = label_distribution["count"] / len(df.index) * 100
    
    for index, row in label_distribution.iterrows():
        print(f"Label in the dataset is '{row['label']}' comprising {row['count']:.2f}% of the dataset.")
        
print_label_counts(train_df)

Label in the dataset is '0.0' comprising 78.80% of the dataset.
Label in the dataset is '1.0' comprising 18.81% of the dataset.
Label in the dataset is '2.0' comprising 2.38% of the dataset.


We can see that the majority of data in the Hate speech version 2 dataset, is labeled as non-offensive -> Normal, with this class representing 78.80% of the data points. So a model that would always predict this, can achieve the minimum accuracy of this. 

In [15]:
from sklearn.metrics import classification_report, accuracy_score

def get_baseline_predictions(validation_df_):
    y_baseline_pred = [0] * len(validation_df_.index)
    print(f"Accuracy: {accuracy_score(validation_df_['label'], y_baseline_pred)}")
    print(f"Classification report:\n{classification_report(validation_df_['label'], y_baseline_pred, zero_division=1)}")

get_baseline_predictions(validation_df)

Accuracy: 0.7909985422740525
Classification report:
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      8682
           1       1.00      0.00      0.00      2050
           2       1.00      0.00      0.00       244

    accuracy                           0.79     10976
   macro avg       0.93      0.33      0.29     10976
weighted avg       0.83      0.79      0.70     10976



Authors report they reached accuracy of 98% on the English dataset.

# Toxify dataset

In [16]:
from src.preprocessing.toxigen_querying import load_toxigen_train_and_validation

# the b'' is present only because the text is in UTF-8 encoding
train_df_toxi, validation_df_toxi = load_toxigen_train_and_validation()
train_df_toxi

Loaded 7168/8960 train samples and 1792/8960 validation samples


,label,text,target_group
0,hate,b'asians are trying to take over the world wit...,asian
1,neutral,b'i have never seen a hotter lineup in my life...,mexican
2,hate,b'muslims deseve to die',muslim
3,neutral,b'people who are deaf or hard of hearing shoul...,physical_dis
4,hate,"b""Jews were all bad people because they're in ...",jewish
...,...,...,...
7163,neutral,"b""i don't think i would be as good at math as ...",chinese
7164,hate,b'theres a HUGE difference between many (i.e. ...,latino
7165,hate,"b'he is from china, but he looks like a white ...",chinese
7166,hate,"b'no matter what they do or how hard they try,...",black


In [17]:
print_label_counts(train_df_toxi)

Label in the dataset is 'neutral' comprising 50.10% of the dataset.
Label in the dataset is 'hate' comprising 49.90% of the dataset.


In [18]:
# map hate to 1, and neutral to 0
validation_df_toxi["label"] = validation_df_toxi["label"].replace({"neutral": 0, "hate": 1})
get_baseline_predictions(validation_df_toxi)

Accuracy: 0.5061383928571429
Classification report:
              precision    recall  f1-score   support

           0       0.51      1.00      0.67       907
           1       1.00      0.00      0.00       885

    accuracy                           0.51      1792
   macro avg       0.75      0.50      0.34      1792
weighted avg       0.75      0.51      0.34      1792



C:\Users\Zan\AppData\Local\Temp\ipykernel_16956\4119098941.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_df_toxi["label"] = validation_df_toxi["label"].replace({"neutral": 0, "hate": 1})
